In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests
from bs4 import BeautifulSoup

In [2]:
url = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book.json"

with open(url,"r",encoding="utf-8") as f:
    data = json.load(f)

In [3]:
data = pd.DataFrame(data)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [5]:
# 책 소개 없는 애들만
x = data.loc[data['책소개'].isna(), ['ISBN']]

In [6]:
x

,ISBN
36,9788962515510
47,9791186492369
95,9788928518357
102,9791162431382
104,9788915001275
...,...
49938,9791169501262
49950,9788968472398
49962,9788956768861
49974,9791136426611


In [7]:
x['ISBN']

,ISBN
36,9788962515510
47,9791186492369
95,9788928518357
102,9791162431382
104,9788915001275
...,...
49938,9791169501262
49950,9788968472398
49962,9788956768861
49974,9791136426611


In [8]:
x.reset_index(drop=True,inplace=True)

In [9]:
x['ISBN'] = x['ISBN'].astype('int64')
# 혹시 몰라서 int로. 32bit되려나

In [10]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ISBN    2998 non-null   int64
dtypes: int64(1)
memory usage: 23.6 KB


In [11]:
x

,ISBN
0,9788962515510
1,9791186492369
2,9788928518357
3,9791162431382
4,9788915001275
...,...
2993,9791169501262
2994,9788968472398
2995,9788956768861
2996,9791136426611


In [18]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f""
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면 None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음")
        return None

In [19]:
x['url'] = x['ISBN'].apply(get_product_url_from_isbn)

ISBN 9788962515510: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000969234
ISBN 9791186492369: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9788928518357: 상품 URL -> https://product.kyobobook.co.kr/detail/S000201463394
ISBN 9791162431382: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001812721
ISBN 9788915001275: 상품 URL -> https://product.kyobobook.co.kr/detail/S000202706008
ISBN 9788962397215: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000966896
ISBN 9791137225138: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060614084
ISBN 9788952785688: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000734814
ISBN 9791137210981: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060611981
ISBN 9791190145657: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001936018
ISBN 9791127296377: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060610785
ISBN 9791160455908: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9791137267169: 

In [20]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [33]:
x['url'].isna().sum()

499